In [1]:
import pandas as pd
import numpy as np
import re
import math
import collections
from datetime import datetime, date, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import warnings;warnings.simplefilter('ignore')
%matplotlib inline

In [2]:
data = pd.read_csv('main_task.csv')

In [3]:
#Функция для получения списка в столбце
def list_for_object (variable):
    result = []
    pattern = re.compile('\dd\s\dd')
    if type(variable) == str:
        list_work = re.sub('\s\'|\'','', variable).split(',')
        for i in list_work:
            var = re.sub('\]','', i)
            var_2 = re.sub('\[','', var)
            var_3 = var_2.lstrip(' ')
            result.append(var_3)
    else: result.append('No_name')
    return result

In [4]:
#функция получения уникальных переменных
def unique_variable(series_data):
    un_var = set() #пустое множество для хранения уникальных значений
    for i in series_data:
        if type(i) == list:
            for j in i:
                un_var.add(j)
        else: un_var.add(i)
    return un_var

In [5]:
#функция для определения распределения уникальных значений
def quantity_un_var(series_data):
    quantity_variable = {} #словарь для хранения информации
    for item in unique_variable(series_data):  # перебираем список кухонь
        quantity_variable[item] = 0 # добавляем в словарь ключ, соответствующий очередной кухне
    for i in series_data:
        if type(i) == list:
            for item in i:   
                quantity_variable[item] += 1
        else: quantity_variable[i] += 1
    return quantity_variable

In [6]:
#функция выбора top данных (90% значению по количеству)
def top_data (series_data):
    #создаем Data Frame для удобства сортировки
    df_quantity_var = pd.DataFrame(data = [quantity_un_var(series_data)])
    df_quantity_var = df_quantity_var.transpose()
    df_quantity_var = df_quantity_var.sort_values(by = 0, ascending = False)
    df_quantity_var_freqs = list(df_quantity_var[0])
    top_quantity_value = int(np.percentile(df_quantity_var_freqs, 90))
    top_var_list = df_quantity_var[df_quantity_var[0] > top_quantity_value]
    index_all_var = df_quantity_var.index
    top_var = list(index_all_var)[:len(top_var_list)-1]
    return top_var

In [7]:
#Функция сбора top данных в строку
def list_for_str (series_data, top_variable):
    s = set()
    for i in series_data:
        if i in top_variable:
            s.add(i)
        else: s.add('other')
    s = list(s)
    result = s[0]
    for j in range(1,len(s)):
        result += ','+s[j]
    return result

In [36]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['Restaurant_id','ID_TA',], axis = 1, inplace=True)
    
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
    df_output['Number of Reviews'].fillna(df_output['Number of Reviews'].mean(), inplace=True)
    df_output['Price Range'] = df_output['Price Range'].fillna(0)
    df_output['Cuisine Style'] = df_output['Cuisine Style'].apply(list_for_object)
    df_output['Reviews'] = df_output['Reviews'].fillna('0')
    
    
    # ################### 3. Encoding ############################################################## 
    # для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
    #df_dummies_city = pd.get_dummies(df_output, columns=[ 'City',], dummy_na=True)
    df_dummies_city = df_output['City'].str.get_dummies()
    df_output = pd.concat([df_output, df_dummies_city],axis=1)
    
    
    
   
    # тут ваш код не Encoding фитчей
    #dummy-переменная столбца Cuisine style
    top_cuisin = top_data(df_output['Cuisine Style'])
    df_output['cuisin_dummy'] = df_output['Cuisine Style'].apply(lambda x: list_for_str(x, top_cuisin))
    df_dummies_cuisin = df_output['cuisin_dummy'].str.get_dummies(',')
    df_output = pd.concat([df_output, df_dummies_cuisin],axis=1)
    
    
    # ################### 4. Feature Engineering ####################################################
    # тут ваш код не генерацию новых фитчей
    ### Создаем признак - количество кухонь в рестаране
    df_output['quantity_cs'] = df_output['Cuisine Style'].apply(lambda x: len(x))
    #Столбец Reviews
    pattern = re.compile('\d\d/\d\d/\d{4}')
    df_output['date_ultimate_reviews'] = df_output['Reviews'].apply(lambda x : pd.to_datetime(re.findall(pattern, x)[0]) if len(re.findall(pattern, x)) >= 1 else None) 
    df_output['date_penultimate_reviews'] = df_output['Reviews'].apply(lambda x : pd.to_datetime(re.findall(pattern, x)[1]) if len(re.findall(pattern, x)) == 2 else None)
    df_output['time_delta'] = (df_output['date_ultimate_reviews'] - df_output['date_penultimate_reviews'])
    df_output['time_delta'] = df_output['time_delta'].apply(lambda x: int(str(x)[:-14]) if len(str(x)) > 5 else 0)
    df_output['time_delta'] = df_output['time_delta'].apply(lambda x: -1 * x if x < 0 else x)
    df_output = df_output.drop(['date_ultimate_reviews', 'date_penultimate_reviews'], axis = 1)
    #Столбец Price Range
    df_output['Price Range'] = df_output['Price Range'].apply(lambda x: 2 if x == 0 else x)
    #Добавление признака: mean_ranking - средний ранг по городам, mean_cuisine - среднее количество кухонь по 
    #городам, mean_timedelta - среднее значение промежутка времени между оставленными отзывами
    city = df_output['City'].unique()
    mean_ranking_city = {}  
    for item in city:  # перебираем список городов
        mean_ranking_city[item] = round(df_output[df_output['City'] == item]['Ranking'].mean(), 3)
    df_output['mean_ranking'] = df_output['City'].replace(to_replace = mean_ranking_city)
    
    mean_cuisine_city = {}  
    for item in city:  # перебираем список городов
        mean_cuisine_city[item] = round(df_output[df_output['City'] == item]['quantity_cs'].mean(), 3)
    df_output['mean_cuisin'] = df_output['City'].replace(to_replace = mean_cuisine_city)
    
    mean_timedelta_city = {}  
    for item in city:  # перебираем список городов
        mean_timedelta_city[item] = round(df_output[df_output['City'] == item]['time_delta'].mean(), 3)
    df_output['mean_timedelta'] = df_output['City'].replace(to_replace = mean_timedelta_city)
    
    
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    return df_output

In [37]:
df_2 = preproc_data(data)

In [38]:
print(df_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 52 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Ranking              40000 non-null  float64
 1   Rating               40000 non-null  float64
 2   Number of Reviews    40000 non-null  float64
 3   Amsterdam            40000 non-null  int64  
 4   Athens               40000 non-null  int64  
 5   Barcelona            40000 non-null  int64  
 6   Berlin               40000 non-null  int64  
 7   Bratislava           40000 non-null  int64  
 8   Brussels             40000 non-null  int64  
 9   Budapest             40000 non-null  int64  
 10  Copenhagen           40000 non-null  int64  
 11  Dublin               40000 non-null  int64  
 12  Edinburgh            40000 non-null  int64  
 13  Geneva               40000 non-null  int64  
 14  Hamburg              40000 non-null  int64  
 15  Helsinki             40000 non-null 

In [ ]:
#df_2['quantity_cs'] = df_2['Cuisine Style'].apply(lambda x: len(x))

In [ ]:
#for i in df_2['Cuisine Style']:
    #print(i)

In [ ]:
#type(df_2['Cuisine Style'][0])

In [ ]:
#print(unique_variable(df_2['Price Range']))

In [ ]:
#print(quantity_un_var(df_2['Price Range']))

In [ ]:
#top_cuisin = top_data(df_2['Cuisine Style'])
#top_cuisin

In [39]:
X = df_2.drop(['Rating'], axis = 1)
y = df_2['Rating']

In [40]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [41]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [42]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [43]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [44]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.2085335
